# Pose Detection - Let's run pose detection of mediapipe

### Input
* Preprocessed image list (inputImages.csv)

### Output
* COCO Result data format [link](https://cocodataset.org/#format-results)

In [23]:
%matplotlib inline
import pandas as pd
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
import pylab
import json
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

#### Load images

In [4]:
inputFile = "./inputImages.csv"
df = pd.read_csv(inputFile)
df

,imageID,filename
0,3156,000000003156.jpg
1,5529,000000005529.jpg
2,9448,000000009448.jpg
3,9483,000000009483.jpg
4,10764,000000010764.jpg
...,...,...
249,568584,000000568584.jpg
250,573258,000000573258.jpg
251,578545,000000578545.jpg
252,578871,000000578871.jpg


#### Run pose detection of mediapipe

In [31]:
kpMapIndex = [
    0, # Nose
    2, # Left eye
    5, # Right eye 
    7, # Left ear
    8, # Right ear
    11, # Left shoulder
    12, # Right shoulder
    13, # Left elbow
    14, # Right elbow
    15, # Left wrist
    16, # Right wrist
    23, # Left hip
    24, # Right hip
    25, # Left knee
    26, # Right knee
    27, # Left ankle
    28  # Right ankle
]

def convertToCOCOKeypoint(imageId, landmarks, width, height):
    try:
        keypoint = []
        total = 0;        
        for mapIdx in kpMapIndex:
            landmark = landmarks.landmark[mapIdx]
            keypoint.append(int(landmark.x * width))
            keypoint.append(int(landmark.y * height))
            keypoint.append(1)
            total += landmark.visibility
        return {
            "image_id" : imageId,
            "category_id" : 1,
            "keypoints" : keypoint,
            "score" : total / 17
        }
    except:
        return {
            "image_id" : imageId,
            "category_id" : 1,
            "keypoints" : [0] * 51,                
            "score" : 0
        }

In [7]:
mp_pose = mp.solutions.pose

In [ ]:
output = []
with mp_pose.Pose(
    static_image_mode=True,
    model_complexity=2,    
    min_detection_confidence=0.5) as pose:
        
    for idx, row in df.iterrows():    
        
        if idx%50 == 0:
            print(idx)
            
        imageId = row['imageID']
        filename = row['filename']
                        
        image = cv2.imread('../images/val2017/'+filename)
        height, width, _ = image.shape        
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mpPose = pose.process(image)
                        
        cocoKeypoint = convertToCOCOKeypoint(imageId, mpPose.pose_landmarks, width, height)                         
        output.append(cocoKeypoint)
        
        #annotatedImage = drawKeypointOnImage(image.copy(), cocoKeypoint["keypoints"])
        
        #cv2.imwrite("./tmp/{}".format(filename).replace(".jpg", "_mp.jpg"), annotatedImage)        
        #plt.imshow(cv2.cvtColor(annotatedImage, cv2.COLOR_BGR2RGB) )
        #plt.show()
    print("Done")

0


#### Save file

In [22]:
with open('./detectedPose.json', 'w') as file:
    json.dump(output, file)

In [29]:
[0] * 3

[0, 0, 0]